# Process BenchExec Experiments

This notebook post-processes the results obtained from Benchexec to generate two tables:

1. A flat table of stats, that can be used for further analysis and plotting.
2. A coverage table per domain and solver, typically reported in papers.

In [ ]:
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
import glob
import re

# sys.dont_write_bytecode = True  # prevent creation of .pyc files

CSV_FOLDER = "stats/july24-redo-benchexec/"
NAME_EXPERIMENT = "cfond"

## 1. Flatten Benchexec CSV tables

Collect all CSV benchexec table result files under `CSV_FOLDER` folder.

In [ ]:
print(f"CSV files found in folder {CSV_FOLDER}:")
files = glob.glob(os.path.join(CSV_FOLDER, "**", "benchmark-*.csv"), recursive=True)

files += glob.glob(os.path.join(CSV_FOLDER, "**", "results.*.table.csv"), recursive=True)

files

Load all CSV data into a single dataframe. Note that each CSV file may include results for many _run sets_, with each having its own (set of) columns.

Each row is the result of a _task_ in the experiment, and every run set has its columns stats for such task. Presumably sets of columns share the same schema.

We then need reshape this structure to have just one set of columns and a new column identifying the run set. So each original row will be expanded into many rows, one per run set.

The first three lines contain header:

1. First line contains the _tool_ used. It starts with `tool` followed by the name of the tool repeated multiple times (to match no. of columns).
2. Second line contains the _runs_ of the experiment. It starts with `run set` and then sets of columns with the name of the runs.
3. Third line contains the _stats_ column names repeated per run set. First column is for name of the task.

In [ ]:
RENAME_COLS = {"benchmarks/benchexe/tasks/": "id", "cputime (s)": "cputime", "walltime (s)": "walltime", "memory (MB)": "memory_mb"}

def get_meta_csv(file):
    """Given a benchexec CSV file, extract the runs (e.g., prp, prp_inv) and how many columns per run"""
    with open(file, "r") as f:
        # first line contains the tool used (repeated one per column needed); e.g.,  PP-FOND
        tools_header = f.readline().split()[1:]
        # second line contains the run/solvers used in the experiment (e.g., prp, prp_inv) and starts with "run set" to be ignored
        runs_header = f.readline().split()[2:]

    runs = set(runs_header)

    no_cols = int(len(runs_header) / len(runs))

    return runs, no_cols


dfs = []
for f in files:
    runs, no_cols = get_meta_csv(f)
    print(f"Runs in file {f}: {runs} with {no_cols} stat columns")

    # go over each set of run columns (a csv file may contain many runs, each with the same columns)
    for k, r in enumerate(list(runs)):
        col_idx = [0] + list(range(k*no_cols + 1, k*no_cols + no_cols + 1))
        print(f"\t Extracting run '{r}' in columns: {col_idx}")

        # read the CSV file from line 3+ (line 3 is header)
        df = pd.read_csv(f, delimiter="\t", skiprows=2, usecols=col_idx)
        df.rename(columns=lambda x: x.split('.')[0], inplace=True)

        df.columns.values[0] = "task"
        # df.rename(columns={df.columns[1]: "task"})

        # populate column run with name of run-solver r
        df.insert(1, 'run', r)
        dfs.append(df)

df_csv = pd.concat(dfs).reset_index(drop=True)

df_csv.rename(columns=RENAME_COLS, inplace=True)

print("runs found:", df_csv["run"].unique())
# df.set_index("task", inplace=True)
df_csv

In [ ]:
# solver/runs found
print("Solvers/run found:", df_csv['run'].unique())

We next **enrich** the dataframe with derived columns:

* domain
* instance
* solver
* solved (boolean)

In [ ]:
def get_benchmark_labels(task_name):
    """From the task description name (e.g., acrobatics_01.yml), extract the benchmark labels, like domain, instance"""
    regex = r"(.+)_([0-9]+)\.yml"

    match = re.match(regex, task_name)
    if match:
        # print(match.groups())
        domain = match.group(1)
        instance = match.group(2)
    else:
        print("Problem extracting labels from task name", task_name)
    return domain, instance

df = df_csv.copy()

# 1 - split task name into domain and instances
df["benchmark"] = df.reset_index()["task"].map(get_benchmark_labels).values
df["domain"] = df["benchmark"].str.get(0)
df["instance"] = df["benchmark"].str.get(1)
df.drop(columns=["benchmark"], inplace=True)

# 2 - map status from benchexec to integers status
map_status = {
    "true": 1,
    "false": 0,
    "True": 1,
    "False": 0,
    False: 0,
    True: 1,
    "OUT OF MEMORY (false)": -2,
    "TIMEOUT (false)": -1,
    "TIMEOUT (true)": 1,
}
df["status2"] = df["status"].map(map_status)

missing_mapping = df[df["status2"].isnull()].shape[0]
if missing_mapping > 0:
    missing_status = [x for x in df["status"].unique() if x not in map_status.keys()]
    print(f"WARNING: {missing_mapping} status values not mapped:", missing_status)
    print(df[df["status2"].isnull()])

df["status"] = df["status2"]
df.drop(columns=["status2"], inplace=True)

# 3 - define Boolean column solved to flag if solved or not based on status
df.insert(3, "solved", df["status"].apply(lambda x: True if x == 1 else False))


# 4 - extract solver from run name
map_solver = {
    "prp.FOND": "prp",
    "paladinus.FOND": "paladinus",
    "fondsat-glucose.FOND": "fondsat-glucose",
    "fondsat-minisat.FOND": "fondsat-minisat",
    "cfondasp1-reg.FOND" : "cfondasp1-reg",
    "cfondasp2-reg.FOND" : "cfondasp2-reg",
    "cfondasp1-fsat" : "asp1-fsat",
    "cfondasp2-fsat" : "asp2-fsat",
}
df["solver"] = df["run"].map(map_solver)

# sanity check status
# df.query("status not in [-1,0,-2,1]")
# df.status = df.status.astype(int) # convert to int
# df.loc[df.status == "OUT OF MEMORY (false)"]
# df.loc[df.status == -1]

# df.dtypes

# note that status should be integer; if float it is bc there must be NaN value!
df

Finally, save all results into a complete CSV file.

In [ ]:
df.to_csv(os.path.join(CSV_FOLDER, f"{NAME_EXPERIMENT}_benchexec_stats.csv"), index=False)

## 2. Coverage Analysis and Table

We now generate **coverage** tables, as they often apper in papers. Basically we compute per benchmark set, domain, and APP type sub-domain, and each solver-run:

- **Coverage:** % of solved instances solved by the solver-run; and
- **Stat metrics:** mean on time, memory usage, and policy size.

In [ ]:
print(df.shape)
df.head()

Calculate % ratio per set/domain/sub_domain/run-solver.

In [ ]:
df_grouped = df.groupby(["domain", "solver"])

#   df_grouped.sum()[["solved"]] = sum all the True instances (sum over bool = number of True)
#   df_grouped.count()[["solved"]] = number of rows in solved column (includes True and Talse values)
df_coverage = df_grouped.sum()[["solved"]] / df_grouped.count()[["solved"]]
df_coverage

Calculate mean metric (for CPU time, memory, and policy size) across the solved instances.

In [ ]:
columns = ["domain", "solver", "cputime", "memory_mb", "policy_size"]
df_solved = df.query("solved == True")[columns]

df_solved_grouped = df_solved.groupby(["domain", "solver"])
df_metrics = df_solved_grouped.mean()
df_metrics

Put together **Coverage** and **Metrics** tables.

In [ ]:
column_names = {
    "solved": "cov",
    "cputime": "time",
    "memory_mb": "mem",
    "policy_size": "size",
}

df_stats = df_coverage.join(df_metrics, how="inner")
df_stats.rename(columns=column_names, inplace=True)

df_stats = df_stats.reset_index()

df_stats

In [ ]:
df_stats_pivot = df_stats.pivot(
    index=["domain"],
    values=["cov", "time", "mem", "size"],
    columns="solver",
)
df_stats_pivot.reset_index(
    inplace=True
)  # unfold multi-index into columns (create integer index)
df_stats_pivot.columns = [
    "_".join(tup).rstrip("_") for tup in df_stats_pivot.columns.values
]

# flat index, but multi-column: 1. coverage / time / policy size and 2. each solver/run
df_stats_pivot = df_stats_pivot.round(2)

df_stats_pivot

Save it to the file, this can be used in the paper.

In [ ]:
df_stats_pivot.to_csv(os.path.join(CSV_FOLDER, f"{NAME_EXPERIMENT}_coverage_table.csv"), index=False)